In [1]:
import numpy as np   # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization
import gc  # garbage collection

import category_encoders as ce  # categorical encoding

In [2]:
# defination the function to optimize the memory usage of the dataframe
def optimize_dataframe_for_memory(df):
 print("current memory usage: ", df.memory_usage().sum() / 1024**2, "MB")

 for col in df.columns:
  _max = df[col].max()
  _min = df[col].min()
  if df[col].dtype  == 'int64':
   if _min > np.iinfo(np.int8).min and _max < np.iinfo(np.int8).max:
    df[col] = df[col].astype(np.int8)
   elif _min > np.iinfo(np.int16).min and _max < np.iinfo(np.int16).max:
    df[col] = df[col].astype(np.int16)
   elif _min > np.iinfo(np.int32).min and _max < np.iinfo(np.int32).max:
    df[col] = df[col].astype(np.int32)
  elif df[col].dtype == 'float64':
   if _min > np.finfo(np.float16).min and _max < np.finfo(np.float16).max:
    df[col] = df[col].astype(np.float16)
   elif _min > np.finfo(np.float32).min and _max < np.finfo(np.float32).max:
    df[col] = df[col].astype(np.float32)
  else:
   continue
 
 print("new memory usage: ", df.memory_usage().sum() / 1024**2, "MB")
 return df

In [3]:
read_path = './data/'
save_path = './results/'
order_products_train = optimize_dataframe_for_memory(pd.read_csv(read_path+"order_products__train.csv"))
order_products_prior = optimize_dataframe_for_memory(pd.read_csv(read_path+"order_products__prior.csv"))
orders = optimize_dataframe_for_memory(pd.read_csv(read_path+"orders.csv"))
products = optimize_dataframe_for_memory(pd.read_csv(read_path+"products.csv"))
aisles = optimize_dataframe_for_memory(pd.read_csv(read_path+"aisles.csv"))
departments = optimize_dataframe_for_memory(pd.read_csv(read_path+"departments.csv"))

current memory usage:  42.25528335571289 MB
new memory usage:  13.204862594604492 MB
current memory usage:  989.8221778869629 MB
new memory usage:  340.25145626068115 MB
current memory usage:  182.70566940307617 MB
new memory usage:  68.51470470428467 MB
current memory usage:  1.5164833068847656 MB
new memory usage:  0.7109184265136719 MB
current memory usage:  0.002170562744140625 MB
new memory usage:  0.00140380859375 MB
current memory usage:  0.000446319580078125 MB
new memory usage:  0.00030612945556640625 MB


In [4]:
prior_orders = orders[orders['eval_set'] == 'prior']
train_orders = orders[orders['eval_set'] == 'train']
test_orders = orders[orders['eval_set'] == 'test']

In [5]:

num_train_orders = len(train_orders)
print("Number of train orders:", num_train_orders)
num_train_unique_order_id = order_products_train['order_id'].nunique()
print("Number of unique order_id in order_products_train:", num_train_unique_order_id)
num_train_products = len(order_products_train)
print("Number of products in order_products_train:", num_train_products)

Number of train orders: 131209
Number of unique order_id in order_products_train: 131209
Number of products in order_products_train: 1384617


In [6]:
prior_unique_user_count = orders[orders['eval_set'] == 'prior']['user_id'].nunique()
prior_nan_count = orders[orders['eval_set'] == 'prior']['days_since_prior_order'].isna().sum()

train_unique_user_count = orders[orders['eval_set'] == 'train']['user_id'].nunique()
train_nan_count = orders[orders['eval_set'] == 'train']['days_since_prior_order'].isna().sum()

test_unique_user_count = orders[orders['eval_set'] == 'test']['user_id'].nunique()
test_nan_count = orders[orders['eval_set'] == 'test']['days_since_prior_order'].isna().sum()

prior_unique_user_count, prior_nan_count, train_unique_user_count, train_nan_count, test_unique_user_count, test_nan_count

(206209, 206209, 131209, 0, 75000, 0)

In [7]:
train_user_ids = train_orders['user_id'].unique()
test_user_ids = test_orders['user_id'].unique()

prior_user_ids = prior_orders['user_id'].unique()

train_in_prior = set(train_user_ids).issubset(prior_user_ids)
test_in_prior = set(test_user_ids).issubset(prior_user_ids)


train_in_prior, test_in_prior

(True, True)

##### step 1:Prepare the data using the Prior dataset. Use an inner join between order_products__prior and orders, keeping only the orders labeled as 'prior'. Then join with the products table

In [8]:
df_prior = pd.merge(order_products_prior, orders, on='order_id', how='inner')
df_prior = pd.merge(df_prior, products, on='product_id', how='left')

In [9]:
df_prior

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32434484,3421083,39678,6,1,25247,prior,24,2,6,21.0,Free & Clear Natural Dishwasher Detergent,74,17
32434485,3421083,11352,7,0,25247,prior,24,2,6,21.0,Organic Mini Sandwich Crackers Peanut Butter,78,19
32434486,3421083,4600,8,0,25247,prior,24,2,6,21.0,All Natural French Toast Sticks,52,1
32434487,3421083,24852,9,1,25247,prior,24,2,6,21.0,Banana,24,4


Build the feature: the times of product user ordered

In [10]:
df_prior['times_product_user_ordered'] = df_prior.groupby(['user_id', 'product_id']).cumcount() + 1

Product level features：

（1）Total unique users of a product： p_users_unique 

（2）Total times the product was ordered： p_orders_total

（3）Total times the product was reordered： p_recorderd_total

（4）Reorder percentage of a product： p_recorderd_percentage

（5）Product's average add-to-cart-order： p_avg_cart

（6）times the product was first ordered: p_order_first_cnt

（7）times the product was second ordered: p_order_second_cnt


In [11]:
dict_agg1 = {
    'user_id': [lambda x: x.nunique()],  
    'reordered': ['count', 'sum', 'mean'],
    'add_to_cart_order': ['mean'],
    'times_product_user_ordered': [lambda x: sum(x == 1), lambda x: sum(x == 2)]
}

prod_f1 = df_prior.groupby('product_id').agg(dict_agg1)


prod_f1.columns = ['p_users_unique', 'p_users_total', 'p_reordered_total', 
                   'p_reordered_percentage', 'p_avg_cart', 
                   'p_order_first_cnt', 'p_order_second_cnt']

prod_f1.reset_index(inplace=True)
prod_f1.head()


,product_id,p_users_unique,p_users_total,p_reordered_total,p_reordered_percentage,p_avg_cart,p_order_first_cnt,p_order_second_cnt
0,1,716,1852,1136,0.613391,5.801836,716,276
1,2,78,90,12,0.133333,9.888889,78,8
2,3,74,277,203,0.732852,6.415162,74,36
3,4,182,329,147,0.446809,9.507599,182,64
4,5,6,15,9,0.600000,6.466667,6,4


In [12]:
prod_f1['p_order_second_percent'] = prod_f1.p_order_second_cnt / prod_f1.p_order_first_cnt

Aisle and department features

（8）Aisle unique users：a_users_unique

（9）Total orders: a_orders_total

（10）Total reorders of a product aisle :a_recorderd_total

（11）Reorder percentage: a_recorderd_percentage

（12）Mean and std of aisle add-to-cart-order: a_avg_cart

In [13]:
dict_agg2 = {
    'user_id':[lambda x: x.nunique()],
    'reordered':['count', 'sum', 'mean'],
    'add_to_cart_order': ['mean'] 
}
ailse_f = df_prior.groupby('aisle_id').agg(dict_agg2)
ailse_f.columns = ['a_users_unique', 'a_users_total', 'a_reordered_total', 
                   'a_reordered_percentage', 'a_avg_cart']
ailse_f.reset_index(inplace=True)
ailse_f.head()

,aisle_id,a_users_unique,a_users_total,a_reordered_total,a_reordered_percentage,a_avg_cart
0,1,20711,71928,42912,0.596597,8.167640
1,2,31222,82491,40365,0.489326,9.275497
2,3,63592,456386,272922,0.598007,9.571935
3,4,53892,200687,98243,0.489533,10.161450
4,5,32312,62510,17542,0.280627,10.297600


In [14]:
depart_f = df_prior.groupby('department_id').agg(dict_agg2)
depart_f.columns = ['d_users_unique', 'd_users_total', 'd_reordered_total', 
                   'd_reordered_percentage', 'd_avg_cart']
depart_f.reset_index(inplace=True)
depart_f.head()

,department_id,d_users_unique,d_users_total,d_reordered_total,d_reordered_percentage,d_avg_cart
0,1,163233,2236432,1211890,0.541885,8.996414
1,2,17875,36291,14806,0.407980,8.277645
2,3,140612,1176787,739188,0.628141,8.084397
3,4,193237,9479291,6160710,0.649913,8.022875
4,5,15798,153696,87595,0.569924,5.428346


merge the products features to a table：

In [15]:
prod_f1 = prod_f1.merge(products, on='product_id', how='left')
prod_f1 = prod_f1.merge(aisles, on='aisle_id', how='left')
prod_f1 = prod_f1.merge(ailse_f, on='aisle_id', how='left')
prod_f1 = prod_f1.merge(depart_f, on='department_id', how='left')
prod_f1 = prod_f1.merge(departments, on='department_id', how='left')
prod_f1.head()

,product_id,p_users_unique,p_users_total,p_reordered_total,p_reordered_percentage,p_avg_cart,p_order_first_cnt,p_order_second_cnt,p_order_second_percent,product_name,...,a_users_total,a_reordered_total,a_reordered_percentage,a_avg_cart,d_users_unique,d_users_total,d_reordered_total,d_reordered_percentage,d_avg_cart,department
0,1,716,1852,1136,0.613391,5.801836,716,276,0.385475,Chocolate Sandwich Cookies,...,234065,128431,0.548698,9.253092,174219,2887550,1657973,0.574180,9.187743,snacks
1,2,78,90,12,0.133333,9.888889,78,8,0.102564,All-Seasons Salt,...,212092,32321,0.152391,9.996181,172755,1875577,650301,0.346721,9.593425,pantry
2,3,74,277,203,0.732852,6.415162,74,36,0.486486,Robust Golden Unsweetened Oolong Tea,...,249341,131556,0.527615,8.519846,172795,2690129,1757892,0.653460,6.976699,beverages
3,4,182,329,147,0.446809,9.507599,182,64,0.351648,Smart Ones Classic Favorites Mini Rigatoni Wit...,...,390299,217262,0.556655,9.207741,163233,2236432,1211890,0.541885,8.996414,frozen
4,5,6,15,9,0.600000,6.466667,6,4,0.666667,Green Chile Anytime Sauce,...,62510,17542,0.280627,10.297600,172755,1875577,650301,0.346721,9.593425,pantry


In [16]:
# prod_f1.drop(['prduct_name', 'aisle_id', 'department_id'], axis=1, inplace=True)

In [17]:
#encoder = ce.BinaryEncoder(cols=['aisle_id', 'department_id'],return_df=True)
#prod_f1 = encoder.fit_transform(prod_f1)

In [18]:
#free some memory
del ailse_f, depart_f, aisles, departments
gc.collect()

0

User level features

(1) u_mean_dow :User's average and std day-of-week of order

(2) u_std_dow 

(3) u_avg_hour :User's average and std hour-of-day of order

(4) u_avg_days_since_prior:User's average and std days-since-prior-order

(5) u_orders_total: Total orders by a user

(6) u_products_unique ：Total unique products user has bought

(7) u_products_total:Total products user has bought

(8) u_reordered_total: user's total reordered products

(9) u_reordered_precent:User's overall reorder percentage

(10) u_avg_order_size：Average order size of a user

(11) u_avg_reordered_orders:User's mean of reordered items of all orders



In [19]:
# when no prior order, the value is null. Imputing as 0
df_prior.days_since_prior_order = df_prior.days_since_prior_order.fillna(0)

In [20]:
dict_agg3 = {
    'order_dow':['mean','std'],
    'order_hour_of_day':['mean','std'],
    'days_since_prior_order':['mean','std'],
    'order_number':[lambda x: x.nunique()],
    'product_id':['count', lambda x: x.nunique()],
    'reordered':['sum', 'mean']
}

u_f = df_prior.groupby('user_id').agg(dict_agg3)
u_f.columns = ['u_mean_dow', 'u_std_dow', 'u_avg_hour', 'u_std_hour', 
                'u_avg_days_since_prior', 'u_std_days_since_prior',
                'u_orders_total', 'u_products_total', 'u_products_unique',
                'u_reordered_total', 'u_reordered_percentage']
u_f.reset_index(inplace=True)
u_f.head()

,user_id,u_mean_dow,u_std_dow,u_avg_hour,u_std_hour,u_avg_days_since_prior,u_std_days_since_prior,u_orders_total,u_products_total,u_products_unique,u_reordered_total,u_reordered_percentage
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559065,10,59,18,41,0.694915
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93,0.476923
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867396,12,88,33,55,0.625000
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1,0.055556
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14,0.378378


In [21]:

u_f2 = df_prior.groupby(['user_id','order_number']).agg({'reordered':['count', 'mean']})
u_f2.columns = ['u_avg_order_size', 'u_avg_reordered_orders']
u_f2.reset_index(inplace=True)
u_f3 = u_f2.groupby(['user_id']).agg({'u_avg_order_size':['mean'],'u_avg_reordered_orders':['mean']})
u_f3.columns = ['u_avg_order_size', 'u_avg_reordered_orders']
u_f3.reset_index(inplace=True)
u_f3.head()


,user_id,u_avg_order_size,u_avg_reordered_orders
0,1,5.900000,0.705833
1,2,13.928571,0.447961
2,3,7.333333,0.658817
3,4,3.600000,0.028571
4,5,9.250000,0.377778


In [22]:
u_f = u_f.merge(u_f3, on='user_id', how='left')
u_f.head()

,user_id,u_mean_dow,u_std_dow,u_avg_hour,u_std_hour,u_avg_days_since_prior,u_std_days_since_prior,u_orders_total,u_products_total,u_products_unique,u_reordered_total,u_reordered_percentage,u_avg_order_size,u_avg_reordered_orders
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559065,10,59,18,41,0.694915,5.900000,0.705833
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93,0.476923,13.928571,0.447961
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867396,12,88,33,55,0.625000,7.333333,0.658817
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1,0.055556,3.600000,0.028571
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14,0.378378,9.250000,0.377778


(12) Percentage of reordered itmes in user's last three orders

(13) Total orders in user's last three orders

In [23]:
u_last_three_orders = u_f2.groupby('user_id')['order_number'].nlargest(3).reset_index()#extract the last three orders from u_f2
u_last_three_orders.head()
u_last_three_orders = u_f2.merge(u_last_three_orders, on=['user_id', 'order_number'], how='inner')# merge the last three orders with u_f2
u_last_three_orders['rank'] = u_last_three_orders.groupby('user_id')['order_number'].rank("dense",ascending=True)
u_last_three_orders_f = u_last_three_orders.pivot_table(index = 'user_id', columns = ['rank'], \
                                                 values=['u_avg_order_size', 'u_avg_reordered_orders']).\
                                                reset_index(drop = False)
u_last_three_orders_f.columns = ['user_id','orders_3', 'orders_2', 'orders_1', 'reorder_3', 'reorder_2', 'reorder_1']
u_last_three_orders_f.head()

,user_id,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,6.0,6.0,9.0,0.666667,1.0,0.666667
1,2,19.0,9.0,16.0,0.578947,0.0,0.625000
2,3,6.0,5.0,6.0,0.833333,1.0,1.000000
3,4,7.0,2.0,3.0,0.142857,0.0,0.000000
4,5,9.0,5.0,12.0,0.444444,0.4,0.666667


In [24]:
u_f = u_f.merge(u_last_three_orders_f, on='user_id', how='left')
u_f.head()

,user_id,u_mean_dow,u_std_dow,u_avg_hour,u_std_hour,u_avg_days_since_prior,u_std_days_since_prior,u_orders_total,u_products_total,u_products_unique,u_reordered_total,u_reordered_percentage,u_avg_order_size,u_avg_reordered_orders,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559065,10,59,18,41,0.694915,5.900000,0.705833,6.0,6.0,9.0,0.666667,1.0,0.666667
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93,0.476923,13.928571,0.447961,19.0,9.0,16.0,0.578947,0.0,0.625000
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867396,12,88,33,55,0.625000,7.333333,0.658817,6.0,5.0,6.0,0.833333,1.0,1.000000
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1,0.055556,3.600000,0.028571,7.0,2.0,3.0,0.142857,0.0,0.000000
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14,0.378378,9.250000,0.377778,9.0,5.0,12.0,0.444444,0.4,0.666667


User and Product level features:

（1）u_p_avg_cart: User's avg add-to-cart-order for a product

（2）u_p_avg_days_since_prior: User's avg days_since_prior_order for a product

（3）u_p_reordered_total,u_p_reordered_percentage: User's product total orders, reorders and reorders percentage

（4）User's order number when the product was bought last



In [25]:
dict_agg5 = {
            'add_to_cart_order': ['mean'],
            'days_since_prior_order': ['mean'],
            'reordered': ['count', 'sum', 'mean'],
            'order_number': ['max']
            }
u_prod_f = df_prior.groupby(['user_id', 'product_id']).agg(dict_agg5)
u_prod_f.columns = ['u_p_avg_cart', 'u_p_avg_days_since_prior', 
                     'u_p_orders_total','u_p_reordered_total', 'u_p_reordered_percentage',
                     'u_p_last_order']
u_prod_f.reset_index(inplace=True)
u_prod_f.head()

,user_id,product_id,u_p_avg_cart,u_p_avg_days_since_prior,u_p_orders_total,u_p_reordered_total,u_p_reordered_percentage,u_p_last_order
0,1,196,1.400000,17.600000,10,9,0.900000,10
1,1,10258,3.333333,19.555555,9,8,0.888889,10
2,1,10326,5.000000,28.000000,1,0,0.000000,5
3,1,12427,3.300000,17.600000,10,9,0.900000,10
4,1,13032,6.333333,21.666666,3,2,0.666667,10


 User's product purchase history of last three orders

In [26]:
u_last_three_orders.head()

,user_id,order_number,u_avg_order_size,u_avg_reordered_orders,level_1,rank
0,1,8,6,0.666667,7,1.0
1,1,9,6,1.000000,8,2.0
2,1,10,9,0.666667,9,3.0
3,2,12,19,0.578947,21,1.0
4,2,13,9,0.000000,22,2.0


In [27]:
last_three_orders = df_prior.merge(u_last_three_orders, on=['user_id', 'order_number'], how='inner')
last_three_orders.head()
last_three_orders['rank'] = last_three_orders.groupby(['user_id','product_id'])['order_number'].rank("dense",ascending=True)

product_last_three_orders = last_three_orders.pivot_table(index = ['user_id', 'product_id'],\
                                                   columns='rank', values = 'reordered').reset_index()
product_last_three_orders.columns = ['user_id', 'product_id', 'is_reorder_3', 'is_reorder_2', 'is_reorder_1']
product_last_three_orders.fillna(0, inplace = True)
product_last_three_orders.head()

,user_id,product_id,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,1.0,1.0,1.0
1,1,10258,1.0,1.0,1.0
2,1,12427,1.0,1.0,1.0
3,1,13032,1.0,0.0,0.0
4,1,25133,1.0,1.0,1.0


In [28]:
u_prod_f = u_prod_f.merge(product_last_three_orders, on=['user_id', 'product_id'], how='left')
u_prod_f.head()
u_prod_f.fillna(0, inplace = True)

Saving all features

In [29]:
prod_f1.to_pickle(save_path+'product_features.pkl')
u_f.to_pickle(save_path+'user_features.pkl')
u_prod_f.to_pickle(save_path+'user_product_features.pkl')